In [1]:
from dotenv import load_dotenv
load_dotenv()
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException, UnexpectedAlertPresentException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from datetime import datetime
import pickle, json, time, os
import regex as re



In [2]:
def gen_searches():
    
    sources = ['rst=sfusat', 'rst=sfwsj', 'rst=sfnyt', 'rst=sfglob', '(rst=ftft or rst=ftcom)']
    companies = ['fds=applc']

    return [f'{source} and {company}' for source in sources for company in companies]

In [7]:
def enter_search(driver, wait, dates, year, search):
    
    print(year)
    print(dates)

    wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@class="pillNoMenu"]'))).click()
    wait.until(EC.element_to_be_clickable((By.XPATH, '//select[@name="dr"]'))).click()
    wait.until(EC.element_to_be_clickable((By.XPATH, '//option[@value="Custom"]'))).click()
    driver.find_element(By.ID, 'frm').send_keys('01')
    driver.find_element(By.ID, 'frd').send_keys('01')
    driver.find_element(By.ID, 'fry').send_keys(year)
    driver.find_element(By.ID, 'tom').send_keys(dates[0])
    driver.find_element(By.ID, 'tod').send_keys(dates[1])
    driver.find_element(By.ID, 'toy').send_keys(year)
    driver.find_element(By.XPATH, '//select[@name="isrd"]').click()
    wait.until(EC.element_to_be_clickable((By.XPATH, '//option[@value="High"]'))).click()
    search_box = driver.find_element(By.XPATH, '//textarea[@name="ftx"]')
    search_box.send_keys(search)
    search_box.send_keys(Keys.ENTER)
    wait.until(EC.visibility_of_element_located((By.XPATH, '//span[@data-channel="Dowjones"]')))
    time.sleep(5)

In [8]:
def get_page_info(soup, counter, year):
    
    article_links = []
    map_dict = {
        'January': '1',
        'February': '1',
        'March': '1',
        'April': '2',
        'May': '2',
        'June': '2',
        'July': '3',
        'August': '3',
        'September': '3',
        'October': '4',
        'November': '4',
        'December': '4',
    }
    
    for headline in soup.find_all('tr', {'class': 'headline'}):
            
        article_links.append(headline.find('a').get('href'))
        sub_list = sum([subtitle.split(' ') for subtitle in headline.find('div').text.split(',') if len(re.findall('[0-9]+', subtitle)) > 0], [])
        month = ''.join(set(sub_list) & set(map_dict.keys()))
        counter[year][map_dict[month]] += 1
        
    return counter, article_links

In [9]:
def get_all_text(eid_username, eid_password, search_function, first_login):
    
    """
    Function takes three arguments:
    
    1. eid_username
    
        - Should be your UT EID username that you use to login to Canvas
    
    2. eid_password
    
        - Should be your password to login to Canvas
        - I recommend you putting the password in a .env file and using the following format for the password argument:
        os.getenv("password_variable_name") where password variable name is a name you choose to define your password variable
        inside of the .env folder.
        
    3. first_login
    
        - A boolean that is True if your browser has not yet logged into Factiva and False otherwise. Every time you
        reopen the browser, you need to set this to True because closing the browser deletes some of your session cookies. Afterwards, you
        may return to the main search page by setting this value to false and rerunning the function.
        
    """
    
    url = 'https://guides.lib.utexas.edu/db/144'
    option = webdriver.ChromeOptions()
    option.add_experimental_option("debuggerAddress", "localhost:9222")
    driver = webdriver.Chrome(executable_path = "C://Users//galon//cd_secure//chromedriver.exe",options=option)
    wait = WebDriverWait(driver, 15)
    driver.set_page_load_timeout(40)
    driver.get(url)
    
    if first_login == True:

        wait.until(EC.element_to_be_clickable((By.ID, 'username')))
        driver.find_element(By.ID, 'username').send_keys(eid_username)
        driver.find_element(By.ID, 'password').send_keys(eid_password)    
        driver.find_element(By.XPATH, "//input[@value='Sign in']").click()
        try:
            wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@id='trust-browser-button']"))).click()
            
        except TimeoutException:
            print('Duo cookies still valid; proceeding to search page...')   
    print(f'Starting at: {datetime.now()}\n')
    
    for search in search_function:
        
        years = range(1995, 2021)
        counter = {year: {'1': 0, '2': 0, '3': 0, '4': 0, 'duplicates': 0} for year in years}
        for year in years:
            
            if year < 2020:
                dates = ('12', '31')
            else:
                dates = ('06', '30')
            enter_search(driver, wait, dates, year, search)

            while year:
                
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                print(f'Soup found! {type(soup)}')
                if "No search results" in soup.find('div', {'id': 'headlines'}).text:
                    driver.get('https://guides.lib.utexas.edu/db/144')
                    break
                else:
                    counter, article_links = get_page_info(soup, counter, year)
                    try:
                        counter['duplicates'] += int(soup.find('span', {'id': 'dedupSummary'}).text.split(':')[1].strip())
                    except KeyError:
                        print('No duplicates on this page')
                    try:
                        wait.until(EC.element_to_be_clickable((By.XPATH, '//a[@class="nextItem"]'))).click()
                        wait.until(EC.visibility_of_element_located((By.XPATH, '//img[@src="../img/listmanager/progress.gif"]')))
                        wait.until(EC.invisibility_of_element_located((By.XPATH, '//img[@src="../img/listmanager/progress.gif"]')))
                        time.sleep(3)
                    except TimeoutException:
                        driver.get('https://guides.lib.utexas.edu/db/144')
                        break

In [ ]:
get_all_text('gal767', os.getenv('eid_password'), gen_searches(), first_login=False)

C:\Users\galon\AppData\Local\Temp\ipykernel_17248\2374334182.py:28: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = "C://Users//galon//cd_secure//chromedriver.exe",options=option)


Starting at: 2022-10-26 19:40:42.859102

1995
('12', '31')
Soup found! <class 'bs4.BeautifulSoup'>
1996
('12', '31')
Soup found! <class 'bs4.BeautifulSoup'>
No duplicates on this page
1997
('12', '31')
Soup found! <class 'bs4.BeautifulSoup'>
No duplicates on this page
1998
('12', '31')
Soup found! <class 'bs4.BeautifulSoup'>
No duplicates on this page
1999
('12', '31')
Soup found! <class 'bs4.BeautifulSoup'>
2000
('12', '31')
Soup found! <class 'bs4.BeautifulSoup'>
No duplicates on this page
2001
('12', '31')
Soup found! <class 'bs4.BeautifulSoup'>
No duplicates on this page
2002
('12', '31')
Soup found! <class 'bs4.BeautifulSoup'>
No duplicates on this page
2003
('12', '31')
Soup found! <class 'bs4.BeautifulSoup'>
No duplicates on this page
2004
('12', '31')
Soup found! <class 'bs4.BeautifulSoup'>
No duplicates on this page
2005
('12', '31')
Soup found! <class 'bs4.BeautifulSoup'>
No duplicates on this page
2006
('12', '31')
Soup found! <class 'bs4.BeautifulSoup'>
No duplicates on thi

In [ ]:
# print(soup.find('div', {'id': 'headlines'}).text)

In [ ]:
def scrape_hospitals():
    
    url = 'https://www.google.com/'   
    option = webdriver.ChromeOptions()
    option.add_experimental_option("debuggerAddress", "localhost:9222")
    driver = webdriver.Chrome(executable_path = "C://Users//galon//cd_secure//chromedriver.exe",options=option)
    wait = WebDriverWait(driver, 15)
    driver.set_page_load_timeout(40)
    driver.get(url)

In [ ]:
# scrape_hospitals()